In [1]:
# Setup Azure OpenAI Agent
from llama_index.llms.azure_openai import AzureOpenAI

api_key = "your_azure_openai_api_key"
azure_endpoint = "https://<your_azure_openai_name>.openai.azure.com/"
api_version = "2023-08-01-preview"

In [2]:
# Import the AzureACASessionToolSpec from llama_index
from llama_index.tools.azure_aca_session import AzureACASessionToolSpec
# Import the ReActAgent
from llama_index.core.agent import ReActAgent

# Create the AzureACASessionToolSpec with the pool_managment_endpoint set to your session management endpoint
aca_session_tool = AzureACASessionToolSpec(pool_managment_endpoint="<your_session_management_endpoint>")

llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="test-gpt-35-deploy",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# Create the ReActAgent and inject the tools defined in the AzureACASessionToolSpec
agent = ReActAgent.from_tools(aca_session_tool.to_tool_list(), llm=llm, verbose=True)

# Test the agent
print(agent.chat("Tell me the current time in Seattle."))


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: code_interpreter
Action Input: {'python_code': "import datetime\nimport pytz\ndatetime.datetime.now(pytz.timezone('America/Los_Angeles')).strftime('%Y-%m-%d %H:%M:%S')"}
Observation: {'$id': '1', 'status': 'Success', 'stdout': '', 'stderr': '', 'result': '2024-04-25 00:25:18', 'executionTimeInMilliseconds': 100}
Thought: I have the current time in Seattle. I'll use the user's language to answer.
Answer: The current time in Seattle is 2024-04-25 00:25:18.
The current time in Seattle is 2024-04-25 00:25:18.


In [3]:
# Upload a sample temperature file of a day in Redmond Washington and ask a question about it
res = aca_session_tool.upload_file(local_file_path="<some_local_path>/TemperatureData.csv")
if len(res) != 0:
    print(agent.chat("What is the highest temperature in the file I uploaded into the home directory?"))


Thought: I need to use a tool to help me answer the question.
Action: list_files
Action Input: {}
Observation: [RemoteFileMetadata(filename='TemperatureData.csv', size_in_bytes=514, file_full_path='/mnt/data/TemperatureData.csv')]
Thought: I can use the pandas library to read the csv file and find the highest temperature.
Action: code_interpreter
Action Input: {'python_code': "import pandas as pd\n\nfile_path = '/mnt/data/TemperatureData.csv'\ndata = pd.read_csv(file_path)\n\nhighest_temp = data['Temperature'].max()\nprint(highest_temp)"}
Observation: {'$id': '1', 'status': 'Success', 'stdout': '12.4\n', 'stderr': '', 'result': '', 'executionTimeInMilliseconds': 1692}
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The highest temperature in the file TemperatureData.csv is 12.4.
The highest temperature in the file TemperatureData.csv is 12.4.


In [4]:
# Ask the LLM to perform modifications on the sample temperature file
print(agent.chat("Rearrange the temperature data in a descending order and save it back to the original csv file."))


Thought: I need to use the list_files tool to check if the file exists in the home directory. Then I'll use the code_interpreter tool to read the csv file and sort the temperature data in descending order. Finally, I'll use the code_interpreter tool to save the sorted data back to the original csv file.
Action: list_files
Action Input: {}
Observation: [RemoteFileMetadata(filename='TemperatureData.csv', size_in_bytes=514, file_full_path='/mnt/data/TemperatureData.csv')]
Thought: The file exists in the home directory. I'll now use the code_interpreter tool to read the csv file and sort the temperature data in descending order.
Action: code_interpreter
Action Input: {'python_code': "import pandas as pd\n\n# Read the csv file\ndf = pd.read_csv('/mnt/data/TemperatureData.csv')\n\n# Sort the temperature data in descending order\nsorted_df = df.sort_values(by=['Temperature'], ascending=False)\n\n# Save the sorted data back to the original csv file\nsorted_df.to_csv('/mnt/data/TemperatureData.

In [5]:
# Download the modified file
aca_session_tool.download_file(remote_file_path="TemperatureData.csv", local_file_path="<some_local_path>/SortedTemperatureData.csv")


In [8]:
# For comparison, print the first 10 lines of the original file
with open("TemperatureData.csv", "r") as f:
    for i in range(10):
        print(f.readline().strip())


Time,Temperature
4/11/2024 0:00,6.5
4/11/2024 1:00,5.2
4/11/2024 2:00,4.8
4/11/2024 3:00,5.1
4/11/2024 4:00,5.7
4/11/2024 5:00,5.1
4/11/2024 6:00,4.5
4/11/2024 7:00,5.5
4/11/2024 8:00,5.3


In [7]:
# For comparison, print the first 10 lines of the sorted file downloaded from session pool
with open("SortedTemperatureData.csv", "r") as f:
    for i in range(10):
        print(f.readline().strip())


Time,Temperature
4/11/2024 20:00,12.4
4/11/2024 19:00,12.3
4/11/2024 17:00,12.3
4/11/2024 18:00,12.1
4/11/2024 16:00,11.7
4/11/2024 15:00,11.3
4/11/2024 21:00,10.9
4/11/2024 22:00,10.0
4/11/2024 23:00,9.4
